In [2]:
%matplotlib inline
from pathlib import *
import pandas as pd
import numpy as np
import os, six
import mne
import nibabel as nib
import math
from surfer import Brain
import statsmodels as sm
from scipy import stats as ss
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))
pd.set_option('display.width', 999999)
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 999
from pylabs.projects.bbc.pairing import dwipairing
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())

In [3]:
contrl_sid = ['sub-bbc209', 'sub-bbc211', 'sub-bbc208', 'sub-bbc202', 'sub-bbc249', 'sub-bbc241', 'sub-bbc243', 'sub-bbc231', 'sub-bbc253']
foster_sid = ['sub-bbc101', 'sub-bbc105', 'sub-bbc106', 'sub-bbc108', 'sub-bbc113', 'sub-bbc116', 'sub-bbc118', 'sub-bbc119', 'sub-bbc120']
lh_ct = pd.DataFrame.from_csv('/Users/mrjeffs/Documents/Research/data/bbc/freesurfer_stats_results/grp_parc_stats_lh_cortical_thickness.csv', index_col=None)
rh_ct = pd.DataFrame.from_csv('/Users/mrjeffs/Documents/Research/data/bbc/freesurfer_stats_results/grp_parc_stats_rh_cortical_thickness.csv', index_col=None)

In [4]:
lh_ct['Subjid'] = lh_ct['lh.aparc.a2009s.thickness'].str.partition('/').drop([2,1], axis=1)
lh_ct = lh_ct.set_index(['Subjid'])
lh_ct = lh_ct.drop('lh.aparc.a2009s.thickness', axis=1)
lh_ct = lh_ct.transpose().astype('float')
rh_ct['Subjid'] = rh_ct['rh.aparc.a2009s.thickness'].str.partition('/').drop([2,1], axis=1)
rh_ct = rh_ct.set_index(['Subjid'])
rh_ct = rh_ct.drop('rh.aparc.a2009s.thickness', axis=1)
rh_ct = rh_ct.transpose().astype('float')

In [5]:
lh_ct_ctrl = lh_ct[contrl_sid]
lh_ct_fost = lh_ct[foster_sid]
lh_ct['209-101'] = lh_ct['sub-bbc209'] - lh_ct['sub-bbc101']
lh_ct['211-105'] = lh_ct['sub-bbc211'] - lh_ct['sub-bbc105']
lh_ct['208-106'] = lh_ct['sub-bbc208'] - lh_ct['sub-bbc106']
lh_ct['202-108'] = lh_ct['sub-bbc202'] - lh_ct['sub-bbc108']
lh_ct['249-113'] = lh_ct['sub-bbc249'] - lh_ct['sub-bbc113']
lh_ct['241-116'] = lh_ct['sub-bbc241'] - lh_ct['sub-bbc116']
lh_ct['243-118'] = lh_ct['sub-bbc243'] - lh_ct['sub-bbc118']
lh_ct['231-119'] = lh_ct['sub-bbc231'] - lh_ct['sub-bbc119']
lh_ct['253-120'] = lh_ct['sub-bbc253'] - lh_ct['sub-bbc120']
lh_paired_sub = lh_ct[['209-101', '211-105', '208-106', '202-108', '249-113', '241-116', '243-118', '231-119', '253-120']]
lh_paired_sub = lh_paired_sub.transpose()

In [6]:
lh_paired_sub

,lh_G&S_frontomargin_thickness,lh_G&S_occipital_inf_thickness,lh_G&S_paracentral_thickness,lh_G&S_subcentral_thickness,lh_G&S_transv_frontopol_thickness,lh_G&S_cingul-Ant_thickness,lh_G&S_cingul-Mid-Ant_thickness,lh_G&S_cingul-Mid-Post_thickness,lh_G_cingul-Post-dorsal_thickness,lh_G_cingul-Post-ventral_thickness,...,lh_S_precentral-inf-part_thickness,lh_S_precentral-sup-part_thickness,lh_S_suborbital_thickness,lh_S_subparietal_thickness,lh_S_temporal_inf_thickness,lh_S_temporal_sup_thickness,lh_S_temporal_transverse_thickness,lh_MeanThickness_thickness,BrainSegVolNotVent,eTIV
Subjid,,,,,,,,,,,,,,,,,,,,,
209-101,0.068,0.114,0.439,0.411,-0.485,0.396,0.522,0.338,0.209,-0.326,...,0.416,0.899,-0.260,0.342,0.764,0.710,0.616,0.35712,12477.0,-263151.20658
211-105,0.675,0.480,0.586,0.159,0.735,0.125,-0.021,-0.012,0.047,0.331,...,0.103,0.395,0.657,0.449,0.194,0.359,-0.598,0.34532,-11609.0,-232468.20298
208-106,0.227,-0.034,0.106,-0.170,0.366,0.594,0.567,0.083,-0.056,-0.052,...,0.253,0.329,0.235,0.057,0.050,0.138,-0.073,0.11919,-104185.0,-101234.60336
202-108,0.009,-0.059,0.190,-0.695,0.172,0.023,0.185,-0.407,-0.565,-0.618,...,0.125,0.222,0.118,-0.291,-0.517,-0.163,-0.644,-0.32367,-48793.0,196476.13841
249-113,0.257,0.041,0.311,-0.157,0.474,0.606,0.057,0.341,-0.102,0.697,...,0.087,0.399,1.286,0.176,0.487,0.169,0.238,0.20067,-96630.0,-3192.59407
241-116,0.066,-0.058,-0.058,0.155,0.488,0.495,-0.016,0.067,-0.286,0.109,...,0.624,-0.252,1.659,-0.407,-0.097,-0.010,-0.844,0.12593,-9823.0,-81100.17617
243-118,0.085,0.097,-0.267,-0.539,0.514,-0.083,-0.194,-0.142,0.141,0.497,...,0.250,0.191,0.148,0.092,0.548,0.027,-0.278,-0.04118,-142993.0,-104728.87876
231-119,-0.015,0.049,-0.427,0.030,-0.322,0.043,0.213,0.293,0.178,-0.028,...,0.013,-0.069,-0.445,0.337,-0.018,0.143,-0.196,-0.03009,38650.0,211436.72274
253-120,-0.261,-0.210,-0.231,-0.416,0.305,0.015,0.200,0.012,0.074,0.244,...,0.088,-0.246,-0.563,-0.178,-0.080,-0.033,-0.905,-0.09477,86442.0,258633.90048


In [7]:
lh_ct_stats = lh_paired_sub.apply(ss.ttest_1samp, axis=0, args=(0.0,)).apply(pd.Series)
lh_ct_stats.columns = ['lh-tstat', 'lh-p-value']
lh_ct_stats.index.name = 'Region'
lh_ct_stats['Region'] = lh_ct_stats.index.str.replace('lh_', '').str.replace('_thickness', '')
lh_sig_results = lh_ct_stats[lh_ct_stats['lh-p-value'] <= 0.05].sort_values(by='lh-p-value')
lh_sig_results.set_index('Region')

,lh-tstat,lh-p-value
Region,,
S_orbital_med-olfact,3.899487,0.004548
S_precentral-inf-part,3.348454,0.010103
S_circular_insula_sup,3.010335,0.016805
G&S_cingul-Ant,2.689453,0.027520
Lat_Fis-post,2.551150,0.034114
G_rectus,2.380454,0.044514


In [8]:
rh_ct['209-101'] = rh_ct['sub-bbc209'] - rh_ct['sub-bbc101']
rh_ct['211-105'] = rh_ct['sub-bbc211'] - rh_ct['sub-bbc105']
rh_ct['208-106'] = rh_ct['sub-bbc208'] - rh_ct['sub-bbc106']
rh_ct['202-108'] = rh_ct['sub-bbc202'] - rh_ct['sub-bbc108']
rh_ct['249-113'] = rh_ct['sub-bbc249'] - rh_ct['sub-bbc113']
rh_ct['241-116'] = rh_ct['sub-bbc241'] - rh_ct['sub-bbc116']
rh_ct['243-118'] = rh_ct['sub-bbc243'] - rh_ct['sub-bbc118']
rh_ct['231-119'] = rh_ct['sub-bbc231'] - rh_ct['sub-bbc119']
rh_ct['253-120'] = rh_ct['sub-bbc253'] - rh_ct['sub-bbc120']
rh_paired_sub = rh_ct[['209-101', '211-105', '208-106', '202-108', '249-113', '241-116', '243-118', '231-119', '253-120']]
rh_paired_sub = rh_paired_sub.transpose()

In [9]:
rh_ct_stats = rh_paired_sub.apply(ss.ttest_1samp, axis=0, args=(0.0,)).apply(pd.Series)
rh_ct_stats.columns = ['rh-tstat', 'rh-p-value']
rh_ct_stats.index.name = 'Region'
rh_ct_stats['Region'] = rh_ct_stats.index.str.replace('rh_', '').str.replace('_thickness', '')
rh_sig_results = rh_ct_stats[rh_ct_stats['rh-p-value'] <= 0.05].sort_values(by='rh-p-value')
rh_sig_results.set_index('Region')

,rh-tstat,rh-p-value
Region,,
S_front_inf,4.827615,0.001309
S_central,4.268221,0.002731
S_precentral-inf-part,4.050218,0.003683
S_orbital-H_Shaped,3.307658,0.010736
G&S_cingul-Mid-Ant,2.943471,0.018611
S_occipital_ant,2.943132,0.018620
S_temporal_inf,2.786596,0.023682
S_front_sup,2.637716,0.029819
G_rectus,2.608000,0.031227


In [10]:
rh_sig_results.merge(lh_sig_results, left_on='Region', right_on='Region', how='outer').fillna('').set_index('Region').sort_values(['rh-p-value', 'lh-p-value'])

,rh-tstat,rh-p-value,lh-tstat,lh-p-value
Region,,,,
S_front_inf,4.82762,0.0013086,,
S_central,4.26822,0.00273076,,
S_precentral-inf-part,4.05022,0.00368309,3.34845,0.0101035
S_orbital-H_Shaped,3.30766,0.0107356,,
G&S_cingul-Mid-Ant,2.94347,0.0186106,,
S_occipital_ant,2.94313,0.0186202,,
S_temporal_inf,2.7866,0.0236823,,
S_front_sup,2.63772,0.0298189,,
G_rectus,2.608,0.0312272,2.38045,0.0445141


In [4]:
import mayavi

In [3]:
os.environ['SUBJECTS_DIR'] = str(fs / 'bbc' / 'reg' / 'ants_vbm_pairedLH_in_template_space')
templ = Brain("template_hires_br_freesurf_v6", 'lh', 'white')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ImportError: Could not import backend for traits
_______________________________________________________________________________
Make sure that you have either the TraitsBackendWx or the TraitsBackendQt
projects installed. If you installed Mayavi with easy_install, try
easy_install <pkg_name>. easy_install Mayavi[app] will also work.

If you performed a source checkout, be sure to run 'python setup.py install'
in Traits, TraitsGUI, and the Traits backend of your choice.

Also make sure that either wxPython or PyQT is installed.
wxPython: http://www.wxpython.org/
PyQT: http://www.riverbankcomputing.co.uk/software/pyqt/intro


In [34]:
file = '/Users/mrjeffs/Documents/Research/data/bbc/reg/ants_vbm_pairedLH_in_template_space/template_hires_br_freesurf_v6/label/lh.aparc.a2009s.annot'
labels = mne.read_labels_from_annot(annot_fname=file, hemi='lh', parc='aparc.a2009s', subject='template_hires_br_freesurf_v6', subjects_dir='/Users/mrjeffs/Documents/Research/data/bbc/reg/ants_vbm_pairedLH_in_template_space',verbose=True)
labels

Reading labels from parcellation..
   read 75 labels from /Users/mrjeffs/Documents/Research/data/bbc/reg/ants_vbm_pairedLH_in_template_space/template_hires_br_freesurf_v6/label/lh.aparc.a2009s.annot
[done]


[<Label  |  template_hires_br_freesurf_v6, u'G&S_cingul-Ant-lh', lh : 2027 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_cingul-Mid-Ant-lh', lh : 1259 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_cingul-Mid-Post-lh', lh : 1451 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_frontomargin-lh', lh : 1167 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_occipital_inf-lh', lh : 1840 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_paracentral-lh', lh : 2351 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_subcentral-lh', lh : 1511 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G&S_transv_frontopol-lh', lh : 908 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G_Ins_lg&S_cent_ins-lh', lh : 915 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G_cingul-Post-dorsal-lh', lh : 696 vertices>,
 <Label  |  template_hires_br_freesurf_v6, u'G_cingul-Post-ventral-lh', lh : 319 vertices>,
 <Label  |  templat

In [25]:
pd_labels.iloc[3][0]

<Label  |  template_hires_br_freesurf_v6, u'G&S_frontomargin-lh', lh : 1167 vertices>

In [33]:
lh_ct_stats.loc['lh_S_oc_middle&Lunatus_thickness']

lh-tstat                 0.849177
lh-p-value               0.420466
Region        S_oc_middle&Lunatus
Name: lh_S_oc_middle&Lunatus_thickness, dtype: object

In [9]:
#paired_sub = paired_sub.groupby('Subjid', axis=0)
#paired_sub.aggregate(ss.ttest_1samp(paired_sub, popmean=0.0, axis=0,))
#paired_sub['mean'] = paired_sub.mean(axis=1)
#paired_sub['std_dev'] = paired_sub.std(axis=1)
#paired_sub['std_err'] = paired_sub['std_dev'] / math.sqrt((len(dwipairing) / 2)-1)

#paired_sub = paired_sub.transpose()
#paired_sub
#lh_ct_tstat, np.sort(lh_ct_pval)
#paired_sub